<a href="https://colab.research.google.com/github/TheLazyCactus/ML_Project/blob/main/Final_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

#url = 'https://raw.githubusercontent.com/TheLazyCactus/ML_Project/refs/heads/main/Extra_data.csv'
#df = pd.read_csv(url, sep=";", low_memory =False)

url = 'https://raw.githubusercontent.com/TheLazyCactus/ML_Project/refs/heads/main/ML_Project_safety.csv'
df = pd.read_csv(url, sep=";", low_memory =False)

Need to change the order to get the oldest value first

In [ ]:
df

,Year,Company code,FAR total,TRIR total,TRIR company only,TRIR contractor only,LTIR total,LTIR company only,LTIR contractor only
486,2014,ZZ,0,"3,61",0,"5,43",0,0,0
456,2014,W,"2,6",NaN,NaN,NaN,"0,37","0,37","0,37"
455,2014,V,0,"1,41","0,54","1,65","0,39","0,11","0,47"
454,2014,U,"7,03","1,97","1,21","2,2","0,49","0,61","0,46"
453,2014,T,0,"4,47","2,57","6,24","0,5","0,51","0,48"
...,...,...,...,...,...,...,...,...,...
31,2023,GG,"1,06","0,38","0,8","0,32","0,18","0,51","0,13"
30,2023,FF,"1,34","0,98","0,75","1,07","0,18","0,15","0,19"
29,2023,EE,0,"0,46","0,33","0,51","0,18","0,33","0,13"
27,2023,CC,0,"0,2",0,"0,24","0,2",0,"0,24"


In [ ]:
df = df.sort_values(by="Year", ascending=True)
#Drop FAR
df.drop("FAR total", axis = 1, inplace=True)

cols = ["TRIR total", "TRIR company only", "TRIR contractor only", "LTIR total", "LTIR company only", "LTIR contractor only" ]  # List of columns to convert
df[cols] = df[cols].replace(',', '.', regex=True).astype(float)

df[cols] = df[cols].astype(float)

df = df.fillna(0)

original_df = df

**EDA**

In [ ]:
print(df.shape)
print(df.columns)
print(df.dtypes)
print(df.nunique())
print(df.isna().sum())

(487, 8)
Index(['Year', 'Company code', 'TRIR total', 'TRIR company only',
       'TRIR contractor only', 'LTIR total', 'LTIR company only',
       'LTIR contractor only'],
      dtype='object')
Year                      int64
Company code             object
TRIR total              float64
TRIR company only       float64
TRIR contractor only    float64
LTIR total              float64
LTIR company only       float64
LTIR contractor only    float64
dtype: object
Year                     10
Company code             59
TRIR total              247
TRIR company only       188
TRIR contractor only    279
LTIR total              121
LTIR company only       105
LTIR contractor only    138
dtype: int64
Year                    0
Company code            0
TRIR total              0
TRIR company only       0
TRIR contractor only    0
LTIR total              0
LTIR company only       0
LTIR contractor only    0
dtype: int64


In [ ]:
df_total = df[["Year", "Company code", "TRIR total", "LTIR total"]]
df_company = df[["Year", "Company code", "TRIR company only", "LTIR company only"]]
df_contractor = df[["Year", "Company code", "TRIR contractor only", "LTIR contractor only"]]

**Time Serie for TRIR**

*ARIMA*

In [ ]:
pip install pmdarima --quiet

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


   ---------------------------------------- 0.0/625.1 kB ? eta -:--:--
    --------------------------------------- 10.2/625.1 kB ? eta -:--:--
   -- ------------------------------------ 41.0/625.1 kB 653.6 kB/s eta 0:00:01
   ------------- -------------------------- 204.8/625.1 kB 1.8 MB/s eta 0:00:01
   -------------------------------- ------- 501.8/625.1 kB 3.2 MB/s eta 0:00:01
   ---------------------------------------- 625.1/625.1 kB 3.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.4/2.8 MB 12.6 MB/s eta 0:00:01
   ------------ --------------------------- 0.8/2.8 MB 10.6 MB/s eta 0:00:01
   ------------------- -------------------- 1.3/2.8 MB 10.6 MB/s eta 0:00:01
   ------------------------- -------------- 1.8/2.8 MB 10.5 MB/s eta 0:00:01
   ------------------------------- -------- 2.2/2.8 MB 10.8 MB/s eta 0:00:01
   -------------------------------------- - 2.7/2.8 MB 10.7 MB/s eta 0:00:01
   ---

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
company_counts = df_total.groupby("Company code")["Year"].count()
# Get companies with 5 or more rows
valid_companies = company_counts[company_counts >= 5].index

# Filter the original DataFrame to keep only those companies
df_total_filtered = df_total[df_total["Company code"].isin(valid_companies)]
df_total_filtered = df_total_filtered.set_index("Year")
original_df = df_total_filtered

In [ ]:
column_list = df_total_filtered["Company code"]

In [ ]:
df_total_filtered

,Year,Company code,TRIR total,LTIR total
456,2014,W,0.00,0.37
455,2014,V,1.41,0.39
454,2014,U,1.97,0.49
453,2014,T,4.47,0.50
452,2014,S,1.29,0.53
...,...,...,...,...
31,2023,GG,0.38,0.18
30,2023,FF,0.98,0.18
29,2023,EE,0.46,0.18
27,2023,CC,0.20,0.20


In [ ]:
df_total_filtered.nunique()

Year             10
Company code     50
TRIR total      243
LTIR total      121
dtype: int64

In [ ]:
df_total_filtered.dtypes

Year              int64
Company code     object
TRIR total      float64
LTIR total      float64
dtype: object

In [ ]:
from joblib import Parallel, delayed
import pmdarima as pm
import pandas as pd

# Function to fit ARIMA and return model summary (for Company only)
def fit_arima(group):
    company = group["Company code"].iloc[0]  # Extract the company name
    ts = group["TRIR total"]  # Convert to time series using the 'TRIR total' column


    try:
        # Fit ARIMA model
        model = pm.auto_arima(ts, start_p=1, start_q=1, max_p=3, max_q=3, d=None,
                              test="adf", seasonal=False, trace=False, error_action="ignore",
                              suppress_warnings=True, stepwise=True)

        print(f"Successfully fitted ARIMA for {company}")  # Add a print statement to verify success
        # Generate predictions and residuals
        preds = model.predict_in_sample()  # In-sample predictions
        residuals = ts.values - preds  # Residuals (actual - predicted)
        return company, model, preds, residuals

    except Exception as e:
        print(f"Error fitting ARIMA for {company}: {str(e)}")
        return company, None, None, None


# Debugging the fitting process and checking the data
#print(df_total_filtered.groupby("Company code")["Year"].count())  # Number of rows per company

from joblib import Parallel, delayed

models = Parallel(n_jobs=-1)(delayed(fit_arima)(group) for _, group in df_total_filtered.groupby("Company code"))

final_df = pd.DataFrame()

for i, company_code in enumerate(df_total_filtered["Company code"].unique()):
    company_df = df_total_filtered[df_total_filtered["Company code"] == company_code]

    # Check if the model produced predictions
    if models[i][2] is None or models[i][3] is None:
        print(f"⚠️ Warning: No predictions for Company {company_code}. Skipping...")
        continue  # Skip this company

    preds = models[i][2].to_frame(name="Predictions")
    residuals = models[i][3].to_frame(name="Residuals")

    # Ensure "Year" is the index
    if "Year" in company_df.columns:
        company_df = company_df.set_index("Year")

    # Debugging: Print shapes before fixing mismatch
    print(f"Company: {company_code}")
    print(f"company_df.shape: {company_df.shape}, preds.shape: {preds.shape}, residuals.shape: {residuals.shape}")

    # Fix length mismatch
    min_len = min(len(company_df), len(preds), len(residuals))
    company_df = company_df.iloc[-min_len:]
    preds = preds.iloc[-min_len:]
    residuals = residuals.iloc[-min_len:]

    # Assign correct index
    preds.index = company_df.index
    residuals.index = company_df.index

    # Concatenate and append
    final_df_rows = pd.concat([company_df, preds, residuals], axis=1)
    final_df = pd.concat([final_df, final_df_rows], axis=0)

print(final_df)



Company: W         
company_df.shape: (10, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: V         
company_df.shape: (10, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: U         
company_df.shape: (9, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: T         
company_df.shape: (9, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: S         
company_df.shape: (9, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: R         
company_df.shape: (10, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: Q         
company_df.shape: (10, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: P         
company_df.shape: (10, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: O         
company_df.shape: (10, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: N         
company_df.shape: (10, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: M         
company_df.shape: (10, 3), preds.shape: (10

In [ ]:
final_df['Predictions'] = final_df['Predictions'].apply(lambda x: max(x, 0))

In [ ]:
final_df

,Company code,TRIR total,LTIR total,Predictions,Residuals
Year,,,,,
2014,W,0.00,0.37,0.000000,4.340000
2015,W,1.41,0.38,6.510021,-2.450021
2016,W,1.17,0.28,3.780000,-1.330000
2017,W,0.76,0.25,0.840000,4.820000
2018,W,3.05,0.28,8.870000,-6.220000
...,...,...,...,...,...
2019,FF,0.84,0.18,1.017046,-0.117046
2020,FF,0.76,0.12,1.384425,0.585575
2021,FF,0.49,0.14,1.902792,-1.292792


In [ ]:
## Fit ARIMA Models for each company group
#models = Parallel(n_jobs=-1)(delayed(fit_arima)(group) for _, group in df_total_filtered.groupby("Company code"))
#
#
#final_df = pd.DataFrame()
#
#
#for i, company_code in enumerate(df_total_filtered["Company code"].unique()):
#    company_df = df_total_filtered[df_total_filtered["Company code"] == company_code]
#    preds = models[i][2]
#    residuals = models[i][3]
#    final_df_rows = pd.concat([company_df, preds, residuals], axis=1, ignore_index=False)
#    print(final_df_rows)
#    preds = preds.to_frame(name="Predictions")  # Assign a column name
#    residuals = residuals.to_frame(name="Residuals")
#    company_df = company_df.set_index("Year")
#    preds = preds.set_index("Year")
#    residuals = residuals.set_index("Year")
#    final_df = pd.concat([final_df, final_df_rows], axis=0)



***LTIR Prediction***

**Train test split without lag**

In [ ]:
# Features for model stacking: TRIR ARIMA predictions and residuals
final_df_reset = final_df.reset_index()
X = final_df_reset[["Year", "TRIR total", "Predictions", "Residuals", "Company code"]]
X = pd.get_dummies(X, columns=["Company code"], drop_first=True)
dummy_columns = X.columns.tolist()
y = final_df_reset["LTIR total"]  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
with open("dummy_columns.pkl", "wb") as file:
    pickle.dump(dummy_columns, file)


**Model selection and evaluation only Model stacking for LTIR total, company and global**



*Random forest*

In [ ]:
#Class model
forest = RandomForestRegressor(n_estimators=200,
                             max_depth=10, random_state=42)
model_forest = forest.fit(X_train, y_train)
pred = forest.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score (Test)", forest.score(X_test, y_test))
print("R2 score (Train)", forest.score(X_train, y_train))

MAE 0.20805955036586077
RMSE 0.3260792068241495
R2 score (Test) 0.23166593389328505
R2 score (Train) 0.8931014652529361


d:\Data_analyst\Ironhack\Anaconda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


*Bagging and Pasting*

In [ ]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=10),
                               n_estimators=100,
                               max_samples=100,
                                random_state=42)#take 80% of the total samples

model_bag = bagging_reg.fit(X_train, y_train)
pred = bagging_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", bagging_reg.score(X_test, y_test))

MAE 0.204528015838353
RMSE 0.3155874066408907
R2 score 0.28031374804877773


d:\Data_analyst\Ironhack\Anaconda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


*Gradient Boosting*

In [ ]:
print(y_train.shape)  # Should print (376,) not (376, 3)


(194,)


In [ ]:

gb_reg = GradientBoostingRegressor(max_depth=10,
                                   n_estimators=100,
                                   random_state=42)
model_gb = gb_reg.fit(X_train, y_train)
pred = gb_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", gb_reg.score(X_test, y_test))
print("R2 score", gb_reg.score(X_train, y_train))

MAE 0.2041383199648172
RMSE 0.3620685158638784
R2 score 0.0527046597867592
R2 score 0.99915642083061


d:\Data_analyst\Ironhack\Anaconda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


*Adaptive Boosting*

In [ ]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=10),
                            n_estimators=100,
                            random_state=42)
model_ada = ada_reg.fit(X_train, y_train)
pred = ada_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", ada_reg.score(X_test, y_test))
print("R2 score", ada_reg.score(X_train, y_train))

MAE 0.21252478815983442
RMSE 0.3263387544679752
R2 score 0.23044231301991092
R2 score 0.9785828010197963


d:\Data_analyst\Ironhack\Anaconda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


**Cross validation**

In [ ]:
from sklearn.model_selection import cross_val_score
#Let's try without lag
X = final_df.drop(columns=['LTIR total'])  # Drop target variable
X = pd.get_dummies(X, columns=["Company code"], drop_first=True)
y = final_df['LTIR total']  # Target variable

# Instantiate the Random Forest Regressor model
#rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Perform cross-validation
cv_results = cross_val_score(model_forest, X, y, cv=5, scoring='neg_mean_absolute_error')

# cv_results returns the negative of the score (since lower MAE is better)
# Convert it back to positive values
cv_results = -cv_results

# Print the results
print(f"Cross-validation results for MAE: {cv_results}")
print(f"Average MAE: {np.mean(cv_results)}")
print(f"Standard Deviation of MAE: {np.std(cv_results)}")

Cross-validation results for MAE: [0.2542018  0.30642994 0.62502668 0.25200593 0.14606892]
Average MAE: 0.31674665415721176
Standard Deviation of MAE: 0.16271270484330935


**Overfitting?**

Compare MAE cross validation to train MAE
If cross validation MAE too high compare to train MAE, model good for training sata but not for new data -> Overfitting

**Hyper param tuning**

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [10, 20, 30],  # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required at each leaf node
    'bootstrap': [True, False]  # Whether bootstrap samples are used when building trees
}

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
# Create the RandomForestRegressor model (No need for DecisionTreeRegressor here)
random_reg = RandomForestRegressor(random_state=42)

# Apply GridSearchCV for hyperparameter tuning
model = GridSearchCV(estimator=random_reg, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)

# Fit the model on your training data
model.fit(X_train, y_train)

# Print the best hyperparameters and the best score
print("Best Hyperparameters:", model.best_params_)
print("Best Score:", model.best_score_)

# Get the best model and evaluate it
best_model = model.best_estimator_


Fitting 5 folds for each of 162 candidates, totalling 810 fits
Best Hyperparameters: {'bootstrap': True, 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
Best Score: 0.4413442920169559


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Assuming you already fitted your GridSearchCV model and obtained best_model
# Get the best model after GridSearchCV
best_model = model.best_estimator_

# Now you can use the trained model to make predictions
pred = best_model.predict(X_test)

# Evaluate the model
print("MAE", mean_absolute_error(y_test, pred))
print("RMSE", mean_squared_error(y_test, pred, squared=False))
print("R2 score (Test)", r2_score(y_test, pred))


MAE 0.19838473936893386
RMSE 0.318937188499496
R2 score (Test) 0.2649545385473334


d:\Data_analyst\Ironhack\Anaconda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
best_model

RandomForestRegressor(max_depth=30, min_samples_split=10, random_state=42)

**Using the model for prediction**

In [ ]:
import pandas as pd
import numpy as np

# Example: Assuming your last year in the dataset is 2023
last_year = 2023
years_to_predict = 5  # Predict for the next 5 years

# Generate the future years
future_years = np.arange(last_year + 1, last_year + 1 + years_to_predict)
future_companies = original_df['Company code'].unique()  # assuming 'Company code' is present

In [ ]:


# Create an empty dataframe for future predictions
future_data = []

# Generate the future dataset for each company and each future year
for company in future_companies:
    for year in future_years:
        future_data.append([company, year])

# Convert the list to a dataframe
future_df = pd.DataFrame(future_data, columns=["Company code", "Year"])





In [ ]:

future_df = pd.get_dummies(future_df, columns=["Company code"], drop_first=True)

# If there are additional features that your model used (e.g., lagged variables), you'll need to add those to the future_df
# For example, if you used lagged values for 'LTIR', you'd need to generate those for the future years
future_df['TRIR total'] = np.nan  # Or you could forecast this too, if needed
future_df['Predictions'] = np.nan  # Fill with NaN or predictions from ARIMA/other models
future_df['Residuals'] = np.nan
# Predict for the next 5 years using the trained model (RandomForestRegressor or similar)
predictions = best_model.predict(future_df)  # Use your trained model here

# Add the predictions to the future dataframe
future_df['Predicted_LTIR'] = predictions

# Print the predictions for the next 5 years
print(future_df)

KeyError: "None of [Index(['Company code'], dtype='object')] are in the [columns]"

In [ ]:
import pickle
from sklearn.linear_model import LinearRegression
import numpy as np



# Save the trained model to a file
with open("LTIR_prediction.pkl", "wb") as file:
    pickle.dump(best_model, file)

print("Model saved as model.pkl")